<a href="https://colab.research.google.com/github/rdemarqui/sentiment_analysis/blob/main/01%20Text%20Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis
## Text Preprocessing

In [1]:
import pandas as pd
import string, os

In [2]:
path = os.getcwd()
path

'/content'

### Load data

In [3]:
# Open dataframe
columns = ['review_text', 'overall_rating', 'recommend_to_a_friend']

try:
  df = pd.read_csv(os.path.join(path, 'data\B2W-Reviews01.zip'),
                   sep=',', compression='zip', usecols=columns)
except:
  !wget https://github.com/rdemarqui/sentiment_analysis/raw/main/data/B2W-Reviews01.zip -q
  df = pd.read_csv(os.path.join(path, 'B2W-Reviews01.zip'),
                   sep=',', compression='zip', usecols=columns)

print(df.shape)
df.head()

(132373, 3)


,overall_rating,recommend_to_a_friend,review_text
0,4,Yes,Estou contente com a compra entrega rápida o ú...
1,4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ..."
2,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...
3,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...
4,5,Yes,"A entrega foi no prazo, as americanas estão de..."


### Normalization

In [4]:
corpus = df.query("overall_rating >= 4 or overall_rating <=2").reset_index(drop=True).copy()
print(corpus.shape)
corpus.head()

(116058, 3)


,overall_rating,recommend_to_a_friend,review_text
0,4,Yes,Estou contente com a compra entrega rápida o ú...
1,4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ..."
2,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...
3,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...
4,5,Yes,"A entrega foi no prazo, as americanas estão de..."


In [5]:
punctuations = list(string.punctuation)
def remove_punctuation(text):
    for item in punctuations:
        text = text.replace(item, " ")

    return text

In [6]:
accentuation = {
    "á": "a", "ã": "a", "à": "a","â": "a",
    "é": "e","ê": "e",
    "í": "i",
    "ó": "o","õ": "o", "ô":"o",
    "ú": "u",
    "ç": "c"
    }
def remove_accentuation(text):
    for item in accentuation.items():
        text = text.replace(str(item[0]), str(item[1]))

    return text

In [7]:
special_charaters = [
    "1","2","3","4","5","6","7","8","9","0",
    " a "," b "," c "," d "," e "," f "," g "," h "," i "," j "," k "," l ",
    " m "," n "," o "," p "," q "," r "," s "," t "," u "," v "," x "," z ",
    "r$", "$"
    ]
def remove_special_characters(text):
    for item in special_charaters:
        text = text.replace(item, " ")

    return text

In [8]:
def normalize_text(df, text_field, lower=True, rem_punct=True, rem_accent=True, rem_spec_caract=True, rem_name=False):
    df[text_field + "_clean"] = df[text_field].astype(str)
    if lower: df[text_field + "_clean"] = df[text_field + "_clean"].str.lower()
    if rem_punct: df[text_field + "_clean"] = df[text_field + "_clean"].apply(remove_punctuation)
    if rem_accent: df[text_field + "_clean"] = df[text_field + "_clean"].apply(remove_accentuation)
    if rem_spec_caract: df[text_field + "_clean"] = df[text_field + "_clean"].apply(remove_special_characters)
    df[text_field + "_clean"] = df[text_field + "_clean"].replace(r'\s+', ' ', regex=True) #remove spaces

In [9]:
normalize_text(corpus, 'review_text')

### Stop Words

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
stops_nltk = nltk.corpus.stopwords.words('portuguese')
stop_words = list(stops_nltk)
stop_words = [remove_accentuation(word) for word in stop_words]
stop_words = list(set(stop_words))

def remove_stop_words(text):
    words_list = text.split()
    words_list = [word for word in words_list if word not in stop_words]
    text = ' '.join(words_list)

    return text

In [12]:
corpus["review_text_clean_stop"] = corpus["review_text_clean"].apply(remove_stop_words)

### Stematization

In [13]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [14]:
stemmer = nltk.stem.RSLPStemmer()
def stematization(text):
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]

    return ' '.join(stemmed_words)

In [15]:
corpus["review_text_clean_stem"] = corpus["review_text_clean"].apply(stematization)

In [16]:
corpus["review_text_clean_stop_stem"] = corpus["review_text_clean_stop"].apply(stematization)

### Lemmatization

In [17]:
import spacy.cli

language_models = ['pt_core_news_sm', 'pt_core_news_md', 'pt_core_news_lg']
spacy_model = language_models[2]

disable = ['tagger', 'parser', 'ner', 'entity_ruler', 'entity_linker', 'textcat',
           'morphologizer','attribute_ruler']

try:
  nlp = spacy.load(spacy_model, disable=disable)
except:
  spacy.cli.download(spacy_model)
  nlp = spacy.load(spacy_model, disable=disable)

print(nlp.pipe_names)

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
['tok2vec', 'lemmatizer']


In [18]:
def lemma_single(text):
    doc = nlp(text)
    lema_words = [token.lemma_ for token in doc]

    return ' '.join(lema_words)

In [19]:
#corpus["review_text_clean_lemma"] = corpus["review_text_clean"].apply(lemma_single)

In [20]:
def lemma_pipe(doc_list):
  lemma_text_list = []
  for doc in nlp.pipe(doc_list, n_process=-1):
      lemma_text_list.append(" ".join(token.lemma_ for token in doc))

  return lemma_text_list

In [21]:
corpus["review_text_clean_lemma"] = lemma_pipe(corpus["review_text_clean"])

In [22]:
corpus["review_text_clean_stop_lemma"] = lemma_pipe(corpus["review_text_clean_stop"])

## Save Data

In [26]:
corpus.head()

,overall_rating,recommend_to_a_friend,review_text,review_text_clean,review_text_clean_stop,review_text_clean_stem,review_text_clean_stop_stem,review_text_clean_lemma,review_text_clean_stop_lemma
0,4,Yes,Estou contente com a compra entrega rápida o ú...,estou contente com compra entrega rapida unico...,contente compra entrega rapida unico problema ...,est cont com compr entreg rap unic problem com...,cont compr entreg rap unic problem americ troc...,estar contente com compra entregar rapido unic...,contente compra entregar rapido unico problema...
1,4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",por apenas eu consegui comprar esse lindo copo...,apenas consegui comprar lindo copo acrilico,por apen eu consegu compr ess lind cop de acril,apen consegu compr lind cop acril,por apenas eu conseguir comprar esse lindo cop...,apenas conseguir comprar lir copo acrilico
2,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,supera em agilidade praticidade outras panelas...,supera agilidade praticidade outras panelas el...,sup em agil pratic outr panel eletr costum us ...,sup agil pratic outr panel eletr costum us out...,superar em agilidade praticidade outro panela ...,superar agilidade praticidade outro panela ele...
3,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,meu filho amou parece de verdade com tantos de...,filho amou parece verdade tantos detalhes,meu filh amou parec de verdad com tant detalh ...,filh amou parec verdad tant detalh,meu filho amar parecer de verdade com tanto de...,filho amar parecer verdade tanto detalhe
4,5,Yes,"A entrega foi no prazo, as americanas estão de...",a entrega foi no prazo as americanas estao de ...,entrega prazo americanas parabens smart tv boa...,a entreg foi no praz as americ esta de parab s...,entreg praz americ parab smart tv boa navegaca...,o entrega ser em o prazo o americana estao de ...,entregar prazo americana parabem smart tv bom ...


In [27]:
filename = 'preprocessed_corpus'
compression_options = dict(method='zip', archive_name=f'{filename}.csv')
corpus.to_csv(f'{filename}.zip', compression=compression_options)

print(f'Preprocessed corpus saved on: {path}')

Preprocessed corpus saved on: /content
